# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [236]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection  import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cerion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cerion\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cerion\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [71]:
dataBasePath = 'sqlite:///Data/DisasterResponse.db'
databaseTableName = 'messagesDataTable'

In [72]:
# load data from database
engine = create_engine(dataBasePath)
df = pd.read_sql(databaseTableName, engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
#X = df[['message','original', 'genre']]
X = df['message']
Y = df.drop(columns=['id', 'message','original', 'genre'])

In [150]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [151]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [152]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(x).lower().strip() for x in tokens]
    return clean_tokens

In [153]:
ost = tokenize('Hello aLl happy worlds')
ost

['hello', 'all', 'happy', 'world']

In [131]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
import re
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [154]:
# pipeline = Pipeline([    
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer()),       
#             ('clf', MultiOutputClassifier(RandomForestClassifier()))])

pipeline = Pipeline([    
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),       
            ('clf', MultiOutputClassifier(RandomForestClassifier()))])
 


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [155]:

#X_train, X_test, Y_train, Y_test = train_test_split(df['message'], Y['related'], test_size=0.2)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [194]:
Y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
17722,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2290,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11857,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
23572,1,0,0,1,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
25682,1,0,0,1,1,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14704,1,0,0,1,1,0,0,0,0,0,...,1,0,1,1,1,1,1,1,0,1
9518,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5427,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22383,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [172]:
Y_pred = pipeline.predict(X_test)

In [ ]:
#Evaluate the model and print the accuracy
def EvaluateModel(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    target_names = [name for name in Y.columns]

    for (name, col) in zip(target_names, range(len(target_names))):
        y_test = Y_test[name].values
        y_pred = Y_pred[:, col]
        print(name)
        print(classification_report(y_test, y_pred))
        print('-'*42)

In [232]:
target_names = [name for name in Y.columns]
#print(Y.columns.shape, Y_test.shape, Y_pred.shape)
for (name, col) in zip(target_names, range(len(target_names))):
    y_test = Y_test[name].values
    y_pred = Y_pred[:, col]
    print(name)
    print(classification_report(y_test, y_pred))
    print('-'*42)

related
              precision    recall  f1-score   support

           0       0.70      0.26      0.38      1211
           1       0.81      0.97      0.88      4002
           2       0.50      0.06      0.11        31

    accuracy                           0.80      5244
   macro avg       0.67      0.43      0.46      5244
weighted avg       0.78      0.80      0.76      5244

------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4379
           1       0.88      0.43      0.58       865

    accuracy                           0.90      5244
   macro avg       0.89      0.71      0.76      5244
weighted avg       0.89      0.90      0.88      5244

------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy        

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4967
           1       0.00      0.00      0.00       277

    accuracy                           0.95      5244
   macro avg       0.47      0.50      0.49      5244
weighted avg       0.90      0.95      0.92      5244

------------------------------------------
weather_related
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      3738
           1       0.88      0.63      0.73      1506

    accuracy                           0.87      5244
   macro avg       0.87      0.79      0.82      5244
weighted avg       0.87      0.87      0.86      5244

------------------------------------------
floods
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4810
           1       0.92      0.31      0.46       434

    accuracy                           0.94      5244
   macro avg       

In [245]:
ost = classification_report(y_test, y_pred)

In [248]:
Y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [251]:
np.array(Y_test) == Y_pred

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [227]:
#Y_test['related'].values
print(Y_test.loc[:,'related'].shape)
print(Y_test['related'].values.shape)

(5244,)
(5244,)


### 6. Improve your model
Use grid search to find better parameters. 

In [233]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000028BB1B5AAF0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [243]:
parameters = {
    #'tfidf__norm':['l1', 'l2'],
    #'clf__estimator__n_estimators':[10,100,1000,5000],
    'clf__estimator__n_estimators':[100,1000],
    'clf__estimator__max_depth':[2,5],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.